In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Value / Hidden number of your choice
value = 243

# Parameters (num_frames / fps = duration of video in seconds)
width, height = 400, 336
num_frames = 90 
text = str(value)
font_path = "arial.ttf"
output_video = f"{value}.avi"
shift_speed = 3 
fps= 30
font_size = 200

# Function to wrap text (only relevant for longer text)
def wrap_text(text, font, max_width):
    lines = []
    words = text.split()
    while words:
        line = ''
        while words and font.getbbox(line + words[0])[2] <= max_width:
            line += (words.pop(0) + ' ')
        lines.append(line)
    return lines

# Generates a random black and white noise background
base_noise = np.random.randint(0, 2, (int(height), int(width)), dtype=np.uint8) * 255
mask_img = Image.new("L", (int(width), int(height)), 0)
draw_mask = ImageDraw.Draw(mask_img)
font = ImageFont.truetype(font_path, font_size)
wrapped_text = wrap_text(text, font, int(width))

# Draw wrapped text
text_mask = np.array(mask_img)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
shift = 0

# Generates frames
for i in range(num_frames):
    shift_x = i * shift_speed  # Horizontal shift
    shift_y = i  # Vertical shift
    
    # Creates a new mask for each frame with different font size
    frame_mask_img = Image.new("L", (int(width), int(height)), 0)
    frame_draw_mask = ImageDraw.Draw(frame_mask_img)
    
    # Scaling: Changes font size (based on frame number)
    if i < num_frames // 2:
        current_font_size = font_size - 2*i  # Increase size
    else:
        current_font_size = font_size - 2*(num_frames - i)  # Decrease size

    current_font = ImageFont.truetype(font_path, int(current_font_size))
    
    # Draws text with current font size 
    y_offset = 10
    for line in wrapped_text:
        bbox = frame_draw_mask.textbbox((0, 0), line, font=current_font)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
        text_x = (int(width) - text_w) + 30
        text_y = y_offset + (int(height) - text_h * len(wrapped_text)) / 2 - 40
        frame_draw_mask.text((text_x, text_y), line, font=current_font, fill=255)
        y_offset += text_h
    
    # Converts mask to array
    frame_text_mask = np.array(frame_mask_img)
    
    # Rolling Background: Axis 1 is horizontal, Axis 0 is vertical
    shifted_bg = np.roll(base_noise, shift_x, axis=1)
    
    # Rolling Text Texture: Axis 1 is horizontal, Axis 0 is vertical
    shifted_text_noise = np.roll(base_noise, shift_y, axis=1)

    # Moves the text mask: Shifts along both axis (vertical and horizontal)
    if i < num_frames // 2:
        shifted_mask = np.roll(frame_text_mask, (-shift_y*2, -shift_y), axis=(0,1))
    else:
        shift = shift + 1
        shifted_mask = np.roll(frame_text_mask, (-shift_y*2 + shift*4, -shift_y + shift*2), axis=(0,1))
    
    # Places shifted noise on text and background
    frame_gray = np.where(
        shifted_mask[..., None] == 255,
        shifted_text_noise[..., None],
        shifted_bg[..., None]
    )
    
    frame_bgr = cv2.cvtColor(frame_gray.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    video.write(frame_bgr)


video.release()
print(f"Video saved as {output_video}")

Video saved as 243.avi


In [ ]:
# Converts video to GIF

import imageio

def convert_video_to_gif(video_path, gif_path, duration_seconds=1, target_fps=30):
    video_reader = cv2.VideoCapture(video_path)
    frames = []
    
    # Calculates frames to capture
    start_time = 0
    end_time = start_time + duration_seconds
    frame_count = 0
    
    while True:
        success, frame = video_reader.read()
        if not success:
            break
            
        current_time = frame_count / video_reader.get(cv2.CAP_PROP_FPS)
        if current_time > end_time:
            break
            
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(rgb_frame)
        frame_count += 1
    
    video_reader.release()
    
    # Saves as a GIF with Infinite Loop
    imageio.mimsave(
        gif_path, 
        frames, 
        fps=target_fps,
        loop=0,  # 0 = Infinite Loop
        duration=1/target_fps 
    )

# Converts the generated video to GIF
output_gif = f"{value}.gif"
convert_video_to_gif(output_video, output_gif)